In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os

for root, dirs, files in os.walk("/content/drive"):
    for d in dirs:
        if "Hasdeo" in d:
            print(os.path.join(root,d))

In [4]:
import os

matches = []

for root, dirs, files in os.walk("/content/drive"):
    for d in dirs:
        if d == "Hasdeo_Training_Dataset":
            matches.append(os.path.join(root,d))

matches

[]

In [5]:
!pip install gdown

In [6]:
import gdown

gdown.download_folder(
    "https://drive.google.com/drive/folders/14tQ7B7Y_LpPTll5CUU6w3mDwlhNREm1-?usp=share_link",
    output="data",
    quiet=False
)

Retrieving folder contents


Processing file 1BIyJFZFQYHD_RWyQQUtjXzC4AjCpT2cF ANN_Hasdeo_Full_2016.tif
Processing file 1ycjY5AS361Czw3j0h_arHCPl7f5W0LpT ANN_Hasdeo_Full_2017.tif
Processing file 198SFCMoCIAY1mnibo7qQcZkBn7N2hoFg ANN_Hasdeo_Full_2018.tif
Processing file 1XkObl0ASYHq0AM3aWuiZ-aRWxgi0fBNK ANN_Hasdeo_Full_2019.tif
Processing file 1Wv6WCn5v8ngowD0YlLn_OzoyPUMpx93T ANN_Hasdeo_Full_2020.tif
Processing file 1gT-YXnu984F8e8J1uqa14YTRjvTrJRrR ANN_Hasdeo_Full_2021.tif
Processing file 10jCbPbvtfGy_Oplj-J2GYdZMdBllf-_2 ANN_Hasdeo_Full_2022.tif
Processing file 10_frXzy0KkKv285F6JWMC1G6Hykixvzk ANN_Hasdeo_Full_2023.tif
Processing file 1j-IE9r1g8xDbHlLOQwQEhqAxwz-Ztsfg ANN_Hasdeo_Full_2024.tif
Processing file 1hAqma9UCp4AdllqPZlj9rCvQxwVDQJiN ANN_Hasdeo_Full_2025.tif
Processing file 197gzqNzlEmrCQKgmNvmwfIzLxGyIUNgX ANN_Kente_Extension_2016.tif
Processing file 1HWjagTJ0dIZAJVx-ej8f6erE3aeC_pkr ANN_Kente_Extension_2018.tif
Processing file 1Vy3gbJrev5ZnLnSLYZZtb_gDelOXOCFr ANN_Kente_Extension_2019.tif
Processing fi

FolderContentsMaximumLimitError: The gdrive folder with url: https://drive.google.com/drive/folders/14tQ7B7Y_LpPTll5CUU6w3mDwlhNREm1-?usp=share_link&hl=en has more than 50 files, gdrive can't download more than this limit.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os

for root, dirs, files in os.walk("/content/drive"):
    for d in dirs:
        if "Hasdeo_Training_Dataset" in d:
            print(os.path.join(root,d))

/content/drive/.shortcut-targets-by-id/14tQ7B7Y_LpPTll5CUU6w3mDwlhNREm1-/Hasdeo_Training_Dataset
/content/drive/MyDrive/Hasdeo_Training_Dataset


In [9]:
input_folder = "/content/drive/MyDrive/Hasdeo_Training_Dataset"

In [10]:
import os
os.listdir(input_folder)[:5]

['MON_Kente_Extension_2020_04_slowdown.tif',
 'MON_Kente_Extension_2020_05_slowdown.tif',
 'MON_Kente_Extension_2020_09_slowdown.tif',
 'MON_Kente_Extension_2020_06_slowdown.tif',
 'MON_Kente_Extension_2021_01_slowdown.tif']

In [11]:
import rasterio
import numpy as np
from PIL import Image
import os

output_folder = "/content/jpg_output"
os.makedirs(output_folder, exist_ok=True)

def stretch(x):
    x = np.nan_to_num(x)
    x_valid = x[x>0]
    if len(x_valid)==0:
        return np.zeros_like(x)

    p2,p98 = np.percentile(x_valid,(2,98))
    if p98-p2 == 0:
        return np.zeros_like(x)

    return np.clip((x-p2)/(p98-p2),0,1)

files = sorted([f for f in os.listdir(input_folder) if f.endswith(".tif")])[:10]

for file in files:

    path = os.path.join(input_folder,file)

    with rasterio.open(path) as src:
        r = src.read(3).astype(float)
        g = src.read(2).astype(float)
        b = src.read(1).astype(float)

    r,g,b = stretch(r), stretch(g), stretch(b)

    rgb = np.dstack((r,g,b))
    rgb = (rgb*255).astype(np.uint8)

    Image.fromarray(rgb).save(f"{output_folder}/{file}.jpg")

    print("Converted:", file)

Converted: ANN_Control_Forest_2016.tif
Converted: ANN_Control_Forest_2018.tif
Converted: ANN_Hasdeo_Full_2016.tif


RasterioIOError: Read failed. See previous exception for details.

In [12]:
import os
import rasterio
import numpy as np
from PIL import Image

# ==============================
# SETTINGS
# ==============================

input_folder = "/content/drive/MyDrive/Hasdeo_Training_Dataset"  # change if needed
output_folder = "/content/jpg_output"
max_images = 10
scale = 4   # 2=high quality, 4=balanced, 8=fastest

os.makedirs(output_folder, exist_ok=True)

# ==============================
# SAFE STRETCH FUNCTION
# ==============================

def stretch(x):
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)

    valid = x[x > 0]
    if len(valid) == 0:
        return np.zeros_like(x)

    p2, p98 = np.percentile(valid, (2, 98))

    if p98 - p2 == 0:
        return np.zeros_like(x)

    return np.clip((x - p2) / (p98 - p2), 0, 1)

# ==============================
# FILE LIST
# ==============================

files = sorted([f for f in os.listdir(input_folder) if f.endswith(".tif")])[:max_images]

print("Processing", len(files), "files\n")

# ==============================
# PROCESS LOOP
# ==============================

failed = []

for file in files:

    path = os.path.join(input_folder, file)

    try:
        with rasterio.open(path) as src:

            r = src.read(3, out_shape=(src.height//scale, src.width//scale))
            g = src.read(2, out_shape=(src.height//scale, src.width//scale))
            b = src.read(1, out_shape=(src.height//scale, src.width//scale))

        r = stretch(r)
        g = stretch(g)
        b = stretch(b)

        rgb = np.dstack((r, g, b))
        rgb = (rgb * 255).astype(np.uint8)

        save_path = os.path.join(output_folder, file.replace(".tif",".jpg"))
        Image.fromarray(rgb).save(save_path)

        print("✔ Converted:", file)

    except Exception as e:
        print("✖ Skipped:", file)
        failed.append((file,str(e)))

# ==============================
# REPORT
# ==============================

print("\nDone\n")

if failed:
    print("Failed Files:")
    for f,e in failed:
        print("-",f)
else:
    print("No failures")

# ==============================
# ZIP OUTPUT
# ==============================

import shutil
shutil.make_archive("converted_images","zip",output_folder)

print("\nZIP ready: converted_images.zip")

Processing 10 files

✔ Converted: ANN_Control_Forest_2016.tif
✔ Converted: ANN_Control_Forest_2018.tif
✔ Converted: ANN_Hasdeo_Full_2016.tif
✔ Converted: ANN_Hasdeo_Full_2017.tif
✔ Converted: ANN_Hasdeo_Full_2018.tif
✔ Converted: ANN_Hasdeo_Full_2019.tif
✔ Converted: ANN_Hasdeo_Full_2020.tif
✔ Converted: ANN_Hasdeo_Full_2021.tif
✔ Converted: ANN_Hasdeo_Full_2022.tif
✔ Converted: ANN_Hasdeo_Full_2023.tif

Done

No failures

ZIP ready: converted_images.zip


In [13]:
import os
import rasterio
import numpy as np
from PIL import Image
import shutil

# ==============================
# SETTINGS
# ==============================

input_folder = "/content/drive/MyDrive/Hasdeo_Training_Dataset"
output_folder = "/content/jpg_output"
scale = 4   # 2=high quality | 4=balanced | 8=fastest

os.makedirs(output_folder, exist_ok=True)

# ==============================
# STRETCH FUNCTION
# ==============================

def stretch(x):
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
    valid = x[x > 0]
    if len(valid) == 0:
        return np.zeros_like(x)

    p2, p98 = np.percentile(valid, (2, 98))
    if p98 - p2 == 0:
        return np.zeros_like(x)

    return np.clip((x - p2) / (p98 - p2), 0, 1)

# ==============================
# GET ALL TIFF FILES
# ==============================

files = sorted([f for f in os.listdir(input_folder) if f.endswith(".tif")])

print("Total TIFF files:", len(files), "\n")

# ==============================
# PROCESS LOOP
# ==============================

failed = []

for i, file in enumerate(files, 1):

    path = os.path.join(input_folder, file)

    try:
        with rasterio.open(path) as src:

            r = src.read(3, out_shape=(src.height//scale, src.width//scale))
            g = src.read(2, out_shape=(src.height//scale, src.width//scale))
            b = src.read(1, out_shape=(src.height//scale, src.width//scale))

        r,g,b = stretch(r), stretch(g), stretch(b)

        rgb = np.dstack((r,g,b))
        rgb = (rgb*255).astype(np.uint8)

        save_path = os.path.join(output_folder, file.replace(".tif",".jpg"))
        Image.fromarray(rgb).save(save_path)

        print(f"[{i}/{len(files)}] ✔ {file}")

    except Exception as e:
        print(f"[{i}/{len(files)}] ✖ Skipped:", file)
        failed.append(file)

# ==============================
# ZIP OUTPUT
# ==============================

zip_path = "converted_images"
shutil.make_archive(zip_path, 'zip', output_folder)

print("\nZIP READY → converted_images.zip")

# ==============================
# FAILURE REPORT
# ==============================

if failed:
    print("\nFailed files:")
    for f in failed:
        print("-",f)
else:
    print("\nAll files processed successfully")

Total TIFF files: 374 

[1/374] ✔ ANN_Control_Forest_2016.tif
[2/374] ✔ ANN_Control_Forest_2018.tif
[3/374] ✔ ANN_Hasdeo_Full_2016.tif
[4/374] ✔ ANN_Hasdeo_Full_2017.tif
[5/374] ✔ ANN_Hasdeo_Full_2018.tif
[6/374] ✔ ANN_Hasdeo_Full_2019.tif
[7/374] ✔ ANN_Hasdeo_Full_2020.tif
[8/374] ✔ ANN_Hasdeo_Full_2021.tif
[9/374] ✔ ANN_Hasdeo_Full_2022.tif
[10/374] ✔ ANN_Hasdeo_Full_2023.tif
[11/374] ✔ ANN_Hasdeo_Full_2024.tif
[12/374] ✔ ANN_Hasdeo_Full_2025.tif
[13/374] ✔ ANN_Kente_Extension_2016.tif
[14/374] ✔ ANN_Kente_Extension_2017.tif
[15/374] ✔ ANN_Kente_Extension_2018.tif
[16/374] ✔ ANN_Kente_Extension_2019.tif
[17/374] ✔ ANN_Kente_Extension_2020.tif
[18/374] ✔ ANN_Kente_Extension_2021.tif
[19/374] ✔ ANN_Kente_Extension_2022.tif
[20/374] ✔ ANN_Kente_Extension_2023.tif
[21/374] ✔ ANN_Kente_Extension_2024.tif
[22/374] ✔ ANN_Kente_Extension_2025.tif
[23/374] ✔ ANN_PEKB_Core_2016.tif
[24/374] ✔ ANN_PEKB_Core_2017.tif
[25/374] ✔ ANN_PEKB_Core_2018.tif
[26/374] ✔ ANN_PEKB_Core_2019.tif
[27/374] ✔ 

In [14]:
from google.colab import files
files.download("converted_images.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>